In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install sentencepiece

#!pip install git+https://github.com/huggingface/transformers.git
!pip install transformers==4.38.2

!pip install accelerate
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install torch==2.2.1
!pip install trl bitsandbytes peft datasets scipy ipywidgets -qU

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Zero-Shot

In [ ]:
model_path = "/content/drive/MyDrive/LLM_to_use/Mistral-7b"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_text_mistral(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    generated_ids = model.generate(encodeds, max_new_tokens=1000, do_sample=False, pad_token_id = eos_token_id)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0].split("[/INST]")[1].rstrip("</s>").strip()

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/peace_analysis/human_annotation_final.csv")
df.head()

,text,Human_annot
0,I think India should give an editing expert to...,0.0
1,F-16 (Pakistan) was shot down by MiG-21 (Indi...,0.0
2,Proud to be Pakistani Alhumduillah Alhumduilla...,0.0
3,India’s journalist are the worst in the world ...,0.0
4,F16 pilot was noob i guess\nMig pilot is pro i...,0.0


In [ ]:
len(df)

10081

In [ ]:
from types import new_class
#df['Mistral_annot'] = pd.Series(dtype='str')
import os
import numpy as np
import gc

prompt = """This is the Definition of De-escalating Content: Expresses positive feelings, unity,
  and a desire for peace between conflicting parties, fostering understanding and cooperation.
  This is the definition of NOT De-escalating Content: Lacks positive feelings, impartiality, or
  promotes harm, contributing to tension and hostility between conflicting parties. Below is an
  instruction that describes a task. Write a response that appropriately completes the request.
  If the text below indicates de-escalating content, answer 1, and if not, answer 0. Do not answer anything except 0 or 1: \n\n"""

new_df = {}

for i in range(len(df)):
  new_df[df['text'][i]] = generate_text_mistral(prompt + df['text'][i])

In [ ]:
'''A comment is marked as hope-speech if it exhibits any of the following: \n
1. mentions that the author comes from a neutral country and exhibits a positive sentiment
 towards both countries in the conflict and doesn’t take a position favoring only one of the conflicting countries.
  \n 2. mentions that the author comes from one of the conflicting countries and exhibits a positive sentiment to an entity
  (all people, media, army, government, specific professionals) of the other country. \n 3. urges fellow citizens to de-escalate,
   to stay calm. \n 4. that the author comes from one of the conflicting countries and criticizes some aspect of the author’s own country.
    \n 5. criticizes some aspect of both conflicting countries. \n 6. urges both countries to be peaceful. \n
    7. talks about the humanitarian cost of war and seeks to avoid civilian casualties. \n
    8. The comment expresses unconditional peace-seeking intent.' +
    "Classify the following text if it is hope speech or not in a 'Yes' or 'No' format: " + f'{df["text"][j]}'))


# Fine-tuned


In [ ]:
import re
def lambda_fn(text, human_annotation):

  text = re.sub(r'[^A-Za-z\s.]', '', text)

  prompt = """This is the Definition of De-escalating Content: Expresses positive feelings, unity,
  and a desire for peace between conflicting parties, fostering understanding and cooperation.
  This is the definition of NOT De-escalating Content: Lacks positive feelings, impartiality, or
  promotes harm, contributing to tension and hostility between conflicting parties. Below is an
  instruction that describes a task. Write a response that appropriately completes the request.
  Use the provided input-response pair to learn if the content in discourse is de-escalating or not.
  If the text talks about de-escalating answer 1 and if not answer 0. Do not answer anything except 0 or 1\n\n"""

  prompt += text

  total_prompt = "<s> [INST]" + prompt + "[/INST]"

  total_prompt = total_prompt + str(human_annotation) + "\n"

  total_prompt = total_prompt + "</s>\n"

  return total_prompt

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/peace_analysis/human_annotation_final.csv')

df['prompt'] = df.apply(lambda x: lambda_fn(x['text'], x['Human_annot']), axis=1)

df = df[['prompt']]

In [ ]:
print(df['prompt'][0])

<s> [INST]This is the Definition of De-escalating Content: Expresses positive feelings, unity,
  and a desire for peace between conflicting parties, fostering understanding and cooperation.
  This is the definition of NOT De-escalating Content: Lacks positive feelings, impartiality, or
  promotes harm, contributing to tension and hostility between conflicting parties. Below is an
  instruction that describes a task. Write a response that appropriately completes the request.
  Use the provided input-response pair to learn if the content in discourse is de-escalating or not.
  If the text talks about de-escalating answer 1 and if not answer 0. Do not answer anything except 0 or 1

I think India should give an editing expert to Pakistan as a Technical gesture [/INST]0.0
</s>



In [ ]:
list_of_dicts = [{'prompt': row['prompt']} for index, row in df.iterrows()]

# Ensure the length of the list is 100 and print the first few elements to verify
print(f"Length of the list: {len(list_of_dicts)}")
print("First few elements:", list_of_dicts[:5])


Length of the list: 10081
First few elements: [{'prompt': '<s> [INST]This is the Definition of De-escalating Content: Expresses positive feelings, unity,\n  and a desire for peace between conflicting parties, fostering understanding and cooperation.\n  This is the definition of NOT De-escalating Content: Lacks positive feelings, impartiality, or\n  promotes harm, contributing to tension and hostility between conflicting parties. Below is an\n  instruction that describes a task. Write a response that appropriately completes the request.\n  Use the provided input-response pair to learn if the content in discourse is de-escalating or not.\n  If the text talks about de-escalating answer 1 and if not answer 0. Do not answer anything except 0 or 1\n\nI think India should give an editing expert to Pakistan as a Technical gesture [/INST]0.0\n</s>\n'}, {'prompt': '<s> [INST]This is the Definition of De-escalating Content: Expresses positive feelings, unity,\n  and a desire for peace between con

In [ ]:
import random
random.seed(42)  # For reproducibility
random.shuffle(list_of_dicts)

split_index = int(len(list_of_dicts) * 0.8)

# Split the list
train_dataset = list_of_dicts[:split_index]
test_dataset = list_of_dicts[split_index:]

# Verify the splits
print(f"Total dataset size: {len(list_of_dicts)}")
print(f"Training dataset size: {len(train_dataset)}")
print(f"Testing dataset size: {len(test_dataset)}")


Total dataset size: 10081
Training dataset size: 8064
Testing dataset size: 2017


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
model_path = "/content/drive/MyDrive/LLM_to_use/Mistral-7b"

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "/content/drive/MyDrive/mistral_instruct_generation",
  # num_train_epochs=2,
  max_steps = 400, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 4,
  warmup_steps = 0.03,
  logging_steps=10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  bf16=True,
  lr_scheduler_type='constant',
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  dataset_text_field='prompt',
  args=args,
  train_dataset=train_dataset,
  eval_dataset=test_dataset
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
20,0.809000,0.735300
40,0.624400,0.634887
60,0.606200,0.618905
80,0.616300,0.612042
100,0.626200,0.607734
120,0.591400,0.604805
140,0.588200,0.602045
160,0.608000,0.600552
180,0.582300,0.598094
200,0.588100,0.596920


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in /content/drive/MyDrive/LLM_to_use/Mistral-7b - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in /content/drive/MyDrive/LLM_to_use/Mistral-7b - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=400, training_loss=0.6071202087402344, metrics={'train_runtime': 3192.3162, 'train_samples_per_second': 0.501, 'train_steps_per_second': 0.125, 'total_flos': 1.4024956713526886e+17, 'train_loss': 0.6071202087402344, 'epoch': 1.91})

In [ ]:
trainer.save_model("/content/drive/MyDrive/peace_instruct_generate_mistral")

Inference

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
device = "cuda"
model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/peace_instruct_generate_mistral",
    device_map='auto',
    #quantization_config=nf4_config,
    #use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/peace_instruct_generate_mistral")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# Your validation data
validation_data = test_dataset

# Process the validation dataset to extract texts and labels
texts = []
labels = []
for item in validation_data:
    texts.append(item['prompt'].split('[/INST]')[0].split('[INST]')[1])  # Remove the last 4 characters (e.g., "0.0</s>")
    labels.append(int(float(item['prompt'].split('[/INST]')[1].split('\n')[0])))  # Extract the label (e.g., 0.0 or 1.0) and convert to int

# Tokenize texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

def generate_text_mistral(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
    generated_ids = model.generate(encodeds, max_new_tokens=1000, do_sample= False, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.batch_decode(generated_ids)
    return decoded[0].split("[/INST]")[1].rstrip("</s>").strip()


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
int(float(generate_text_mistral(texts[0]).split("\n")[0]))

1

In [ ]:
pred_labels = []
for i in range(len(test_dataset)):
    pred_labels.append(int(float(generate_text_mistral(texts[i]).split("\n")[0])))


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# Calculate accuracy score
accuracy = accuracy_score(labels, pred_labels)
# Calculate confusion matrix
conf_matrix = confusion_matrix(labels, pred_labels)

# Calculate F1 score
f1 = f1_score(labels, pred_labels)
print('accuracy:', accuracy)
print('conf_matrix', conf_matrix)
print('f1', f1)

accuracy: 0.9196826970748636
conf_matrix [[1434   98]
 [  64  421]]
f1 0.8386454183266931
